In [1]:
import os

In [2]:
%pwd

'c:\\projects\\sms_spam\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\projects\\sms_spam'

In [5]:
import pandas as pd

In [18]:
df=pd.read_csv("artifacts/data_ingestion/spam_classifier.csv")
df.head()

,Target,Text,no_of_charecters,no_of_words,no_of_sentences,Transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",111,24,2,go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,29,8,2,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,37,2,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,49,13,1,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",61,15,1,nah think goe usf live around though


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5169 entries, 0 to 5168
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Target            5169 non-null   int64 
 1   Text              5169 non-null   object
 2   no_of_charecters  5169 non-null   int64 
 3   no_of_words       5169 non-null   int64 
 4   no_of_sentences   5169 non-null   int64 
 5   Transformed_text  5160 non-null   object
dtypes: int64(4), object(2)
memory usage: 242.4+ KB


In [20]:
df.isnull().sum()

Target              0
Text                0
no_of_charecters    0
no_of_words         0
no_of_sentences     0
Transformed_text    9
dtype: int64

In [9]:
df.shape

(5572, 2)

In [10]:
df.columns

Index(['Target', 'Text'], dtype='object')

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [13]:
from src.spam_classifier.constants import *
from src.spam_classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [15]:
import os
from src.spam_classifier import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [21]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [22]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-03-13 21:19:13,059: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-13 21:19:13,072: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-13 21:19:13,075: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-13 21:19:13,084: INFO: common: created directory at: artifacts]
[2024-03-13 21:19:13,086: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-13 21:19:13,240: INFO: 2148425976: Splited data into training and test sets]
[2024-03-13 21:19:13,246: INFO: 2148425976: (3876, 6)]
[2024-03-13 21:19:13,246: INFO: 2148425976: (1293, 6)]
(3876, 6)
(1293, 6)
